In [1]:
from bac.simulate.namd import (Simulation, NonBondedController, PME, BondConstraint, AtomConstraint, 
                               HarmonicConstraint, non_bonded_controller, VelocityRescaling, VelocityReassignment,
                               LangevinDynamics, BerendsenPressureCoupling)

# Minimization steps.
Minimization of the system is composed of 11 steps, with increseangly released constraints.

In [2]:
integrator = VerletIntegrator(timestep=2.0)


minimize = Simulation(timestep=2.0, number_of_steps=100, temperature=300)
minimize.minimization = True

minimize.parameter_type_CHARMM = False
minimize.amber = True
minimize.parameters = 'build/complex.top'
minimize.coordinates = 'build/complex.pdb'
minimize.read_exclusions = False

minimize.binary_output = False
minimize.name = 'minimize_0'
minimize.output_name = 'minimize_0'
minimize.output_energies = 500
minimize.output_pressure = 500

In [3]:
nbc = NonBondedController()
nbc.excluded_interactions = non_bonded_controller.Interaction.scaled_one_four
nbc.one_four_scaling = 0.833333
nbc.cutoff = 12
nbc.switching = True
nbc.switching_distance = 10
nbc.pairlist_distance = 13.5

nbc.non_bonded_frequency = 1
nbc.full_elect_frequency = 2
nbc.steps_per_cycle = 10

nbc.pme = PME(grid_spacing=1.0)

nbc.wrap_water, nbc.wrap_all = True, True
nbc.cell_basis_vector_1 = 72.449, 0.000,  0.000
nbc.cell_basis_vector_2 =  0.000,93.247,  0.000
nbc.cell_basis_vector_3 =  0.000, 0.000, 79.185
nbc.cell_origin = -0.248, -0.156, -0.047

minimize.non_bonded_controller = nbc

In [4]:
minimize.constraints.bond_constraint = BondConstraint(bonds='all', tolerance=0.00001, iterations=100)

harm_const = HarmonicConstraint(exponent=2, reference_position_file='build/complex.pdb')
harm_const.force_constant_file = 'constraint/f4.pdb'
harm_const.force_constant_column = 'B'
harm_const.scaling = 10
minimize.constraints.harmonic_constraint = harm_const

In [5]:
minimizations = [minimize]

for scaling, index, n_steps in zip([10*0.5**index for index in range(1, 10)] + [0], range(1, 11), [100]*9+[5000]):
    minim = next(minimizations[-1])
    minim.name = f'minimize_{index}'
    minim.output_name = f'minimize_{index}'
    minim.constraints.harmonic_constraint.scaling = scaling
    minim.number_of_steps = n_steps
    minimizations.append(minim)

# Equilibration NVT

In [6]:
equilibrate_1 = next(minimize)
equilibrate_1.name = 'eq1'
equilibrate_1.output_name = 'eq1'
equilibrate_1.minimization = False

# 5000 -> 1000
equilibrate_1.number_of_steps = 1000

equilibrate_1.temperature_controller = VelocityReassignment(frequency=100, increment=1, hold_at=300)

equilibrate_1.restart_frequency = 50000
equilibrate_1.non_bonded_controller.xst_frequency = 5000
equilibrate_1.dcd_frequency = 5000

# Equilibration NPT

In [7]:
equilibrate_2 = next(equilibrate_1)
equilibrate_2.name = 'eq2_0'
equilibrate_2.output_name = 'eq2_0'
equilibrate_2.timestep = 1.0
equilibrate_2.constraints.bond_constraint.bonds = 'water'
equilibrate_2.temperature_controller = LangevinDynamics(temperature=300, damping=5, applies_to_hydrogen=False)
equilibrate_2.pressure_controller = BerendsenPressureCoupling(target=1, compressibility=4.57e-5, relaxation_time=100, 
                                                              frequency=2, use_group_pressure=True)

equilibrate_2.restart_frequency = 5000

/Users/kristofarkas/Developer/BAC2/bac/utils/decorators.py:99: UserWarning: 'BerendsenPressureCoupling' object has no attribute 'simulation'
  warnings.warn(f'{e}')


In [8]:
equilibrations = [equilibrate_2]

for scaling, x in zip([10*0.5**x for x in range(1, 10)] + [0], range(1, 11)):
    eq = next(equilibrations[-1])
    eq.name = f'eq2_{x}'
    eq.output_name = f'eq2_{x}'
    eq.constraints.harmonic_constraint.scaling = scaling
    equilibrations.append(eq)

# Production

In [9]:
production = next(equilibrations[-1])
production.name = 'production'
production.output_name = 'production'
production.timestep = 2.0
production.constraints.bond_constraint.bonds = 'all'
production.constraints.harmonic_constraint = None
production.restart_frequency = 200000


# Workflow

This is where Radical could fit in nicely

In [10]:
from bac.simulate import Workflow, Replica

In [11]:
wf = Workflow(resource='bw', r_dir='esmacs_bw')

In [12]:
wf.simulations = minimizations + [equilibrate_1] + equilibrations + [production]

In [13]:
wf.ensembles = [Replica(number_of=5)]

In [10]:
# This command created the input files and puts them in a folder called esmacs_bw.
wf.preprocess_simulations()